In [ ]:
# test game dengan arsitektur network yang telah ditrain
import sys, pygame
from de
from scipy.spatial import distance
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import array
import math
from sklearn.preprocessing import normalize
#w asal impor aja ini, kali aja scipy misal punya tools kaya math, tapi yasudahlah
from random import randint

pygame.init()

size = width, height = 480, 520
black = 255,255,255
ball_speed = [2,2]
block_speed = [randint(-2,2),0]

ball = pygame.image.load("ball.png")
ball = pygame.transform.scale(ball,(width//32,height//32))
ballrect = ball.get_rect().move([randint(0,width),randint(10,height)])

block = pygame.image.load("degea.png")
block = pygame.transform.scale(block,(width//8,height//8))
blockrect = block.get_rect().move([width//2,0])

field = pygame.image.load("field.jpg")
field = pygame.transform.scale(field,(width,height))

screen = pygame.display.set_mode(size)

ballrect = ball.get_rect().move([randint(0,width),randint(10,height)])
blockrect = block.get_rect().move([width//2,0])

range_ball = 0
score = 0
award = 0
touch = False
hit_wall = False


while 1:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            sys.exit()
    if touch==True:
        print("ouch")
        sys.exit()

    ballrect = ballrect.move(ball_speed)
    blockrect = blockrect.move(block_speed)
    
    ballx = (ballrect.right+ballrect.left)//2
    bally = (ballrect.top+ballrect.bottom)//2
    blockx = (blockrect.right+blockrect.left)//2
    blocky = (blockrect.top+blockrect.bottom)//2
    dx=ballx-blockx
    dy=bally-blocky
    
    dxn = dx / np.linalg.norm(width)
    dyn = dy / np.linalg.norm(height)
    
    range_ball = distance.euclidean((ballx,bally),(blockx,blocky))
    
    if ballrect.left < 0 or ballrect.right > width:
        ball_speed[0] = -ball_speed[0]
    if ballrect.top < 0:
        ball_speed[1] = -ball_speed[1]
        award += -10
        touch=True
    if ballrect.bottom > height:
        ball_speed[1] = -ball_speed[1]
    if (bally < (blockrect.bottom+blockrect.top)//2) and (ballx < blockrect.right) and (ballx > blockrect.left):
        ball_speed[1] = -ball_speed[1]
        score += 10
        award += 10
    if (bally < (blockrect.bottom+blockrect.top)//2) and (ballx > blockrect.left and ballx< blockrect.right):
        ball_speed[0] = -ball_speed[0]
        score += 10
        award += 10
        
   
    font = pygame.font.Font(None, 20)
    text = font.render("fail:"+str(touch)+"Cikk "+str(score)+" Award:"+str(award) + " touch:"+"{0:.2f}".format(range_ball), 1, (10, 10, 10)) # buat cek +" posisi bola:"+str((ballrect.right+ballrect.left)/2)+ " posisi meja:"+str(blockrect.right)
    textpos = text.get_rect(centery=screen.get_height()-10) #centerx=screen.get_width()//2
    
    leftwall=ballx
    rightwall=width-ballx
    topwall=bally
    if((ball_speed[0]<0 and leftwall<topwall)or(ball_speed[0]>0 and rightwall<topwall)):
        hit_wall=1
    else:
        hit_wall=0
    
    a=(ballrect[0]+1)/2
    b=(ballrect[1]+1)/2
    c=np.array([([range_ball],[a],[b],[dxn],[dyn],[hit_wall],[ball_speed[0]],[blockrect[0]])])
    
    predictions= model.predict(c)
    block_speed[0] = (np.argmax(predictions[0])*4)-2
    print(block_speed[0])
    
    screen.fill(black)
    screen.blit(field, [0,0])
    screen.blit(text, textpos)
    screen.blit(ball, ballrect)
    screen.blit(block, blockrect)
    pygame.display.flip()